In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import StandardScaler

import cmdstanpy
from cmdstanpy import CmdStanModel

In [11]:
df = pd.read_csv("df_reduced.csv")

In [12]:
df.shape

(1021, 19)

In [13]:
df.columns

Index(['CAI', 'Trigliceridi', 'Colesterolo_Hdl', 'Glucosio', 'PMAX', 'BMI',
       'Alanina_aminotransferasi_alt', 'Colesterolo_totale',
       'Distribuzione_di_volume', 'Ematocrito_hct', 'Eosinofili_perc',
       'Leucociti_wbc', 'Linfociti_perc', 'Monociti_perc', 'Piastrine',
       'Polso', 'Proteine_totali', 'Volume_medio', 'Eta'],
      dtype='object')

In [14]:
stan_code = r"""
data {
  int<lower=1> N;                       // total observations
  int<lower=1> I;                       // number of donors
  int<lower=1> K;                       // number of target variables (K = 5)
  int<lower=1> P;                       // number of covariates
  int<lower=1> M;                       // Truncation level for DP (e.g., 10 or 15)

  array[N] int<lower=1, upper=I> id;    // donor index for each observation
  matrix[N, K] Y;                       // log target variables
  matrix[N, P] X;                       // covariate matrix
}

parameters {
  // Fixed effects
  matrix[P, K] beta;

  // Random Effects via Dirichlet Process
  matrix[I, K] b;

  array[M] vector[K] mu_cluster;

  vector<lower=0>[K] tau_cluster;
  cholesky_factor_corr[K] L_Omega_cluster;

  // Stick-Breaking parameters
  vector<lower=0, upper=1>[M-1] v;
  real<lower=0> alpha;

  // Residual covariance
  vector<lower=0>[K] tau_eps;
  cholesky_factor_corr[K] L_Omega_eps;
}

transformed parameters {
  vector[M] log_theta; // Log-weights
  {
    real log_remaining = 0; // log(1)
    for (m in 1:(M-1)) {
      log_theta[m] = log(v[m]) + log_remaining;
      log_remaining += log1m(v[m]);
    }
    log_theta[M] = log_remaining;
  }

  matrix[K, K] L_Sigma_cluster = diag_pre_multiply(tau_cluster, L_Omega_cluster);
  matrix[K, K] L_Sigma = diag_pre_multiply(tau_eps, L_Omega_eps);
}

model {
  // Prior for fixed effects
  to_vector(beta) ~ normal(0, 2);

  // Dirichlet Process Prior
  alpha ~ gamma(2, 4);
  v ~ beta(1, alpha);

  for (m in 1:M) {
    mu_cluster[m] ~ normal(0, 2);
  }
  tau_cluster ~ normal(0, 0.5) T[0, ];
  L_Omega_cluster ~ lkj_corr_cholesky(4);

  // DP Mixture Prior on Random Effects b
  for (i in 1:I) {
    vector[M] lps;
    for (m in 1:M) {
      lps[m] = log_theta[m] + multi_normal_cholesky_lpdf(b[i] | mu_cluster[m], L_Sigma_cluster);
    }
    target += log_sum_exp(lps);
  }

  // Likelihood
  tau_eps ~ normal(0, 0.5) T[0, ];
  L_Omega_eps ~ lkj_corr_cholesky(4);

  for (n in 1:N) {
    row_vector[K] mu_n = X[n] * beta + b[id[n]];
    Y[n] ~ multi_normal_cholesky(mu_n, L_Sigma);
  }
}

generated quantities {
  vector[N] log_lik;
  matrix[N, K] Y_rep;
  matrix[N, K] mu;

  cov_matrix[K] Sigma_cluster = tcrossprod(L_Sigma_cluster);
  cov_matrix[K] Sigma_eps = tcrossprod(L_Sigma);
  simplex[M] theta = exp(log_theta);

  for (n in 1:N) {
    row_vector[K] mu_n_row = X[n] * beta + b[id[n]];
    vector[K] mu_n_vec = mu_n_row';
    mu[n] = mu_n_row;

    log_lik[n] = multi_normal_cholesky_lpdf(Y[n]' | mu_n_vec, L_Sigma);
    Y_rep[n] = multi_normal_cholesky_rng(mu_n_vec, L_Sigma)';
  }
}
"""

stan_file = "Model_7.stan"
with open(stan_file, "w") as f:
    f.write(stan_code)

print(f"Stan model written to: {stan_file}")
print("Compiling...")
model = CmdStanModel(stan_file=stan_file)
print("Model compiled.")

11:31:34 - cmdstanpy - INFO - compiling stan file /Users/eli/Desktop/BS_project/Model_7.stan to exe file /Users/eli/Desktop/BS_project/Model_7


Stan model written to: Model_7.stan
Compiling...


11:31:47 - cmdstanpy - INFO - compiled model executable: /Users/eli/Desktop/BS_project/Model_7


Model compiled.


In [15]:
ID_COL = "CAI"
ADD_INTERCEPT = False
target_list = ['PMAX', 'Glucosio', 'Trigliceridi', 'Colesterolo_Hdl', 'BMI']
covariate_cols = df.columns.drop(list(target_list) + [ID_COL])

cols_needed = [ID_COL] + target_list + list(covariate_cols)
df_model = df[cols_needed].dropna().copy()

Y_mat = df_model[target_list].to_numpy(dtype=float)
N, K = Y_mat.shape
X_mat = df_model[covariate_cols].to_numpy(dtype=float)
_, P = X_mat.shape

donor_ids, id_index = np.unique(df_model[ID_COL].to_numpy(), return_inverse=True)
I = len(donor_ids)
id_stan = id_index + 1

stan_data = {
    "N": int(N),
    "I": int(I),
    "K": int(K),
    "P": int(P),
    "Y": Y_mat,
    "X": X_mat,
    "id": id_stan,
    "M": 20
}

In [16]:
fit = model.sample(
    data=stan_data,
    chains=4,
    parallel_chains=4,
    iter_warmup=1500,
    iter_sampling=1000,
    adapt_delta=0.9,
    max_treedepth=12,
    show_progress=True
)
print(fit.diagnose())

11:31:48 - cmdstanpy - INFO - CmdStan start processing




chain 1:   0%|                               | 0/2500 [00:00<?, ?it/s, (Warmup)]




chain 2:   0%|                               | 0/2500 [00:00<?, ?it/s, (Warmup)]





chain 3:   0%|                               | 0/2500 [00:00<?, ?it/s, (Warmup)]






chain 4:   0%|                               | 0/2500 [00:00<?, ?it/s, (Warmup)]




chain 2:   4%|▊                  | 100/2500 [03:58<1:35:15,  2.38s/it, (Warmup)]





chain 3:   4%|▊                  | 100/2500 [04:10<1:40:04,  2.50s/it, (Warmup)]




chain 2:   8%|█▋                   | 200/2500 [04:23<43:17,  1.13s/it, (Warmup)]




chain 2:  12%|██▌                  | 300/2500 [04:36<24:38,  1.49it/s, (Warmup)]





chain 3:   8%|█▋                   | 200/2500 [04:36<45:26,  1.19s/it, (Warmup)]






chain 4:   4%|▊                  | 100/2500 [04:41<1:52:45,  2.82s/it, (Warmup)]




chain 2:  16%|███▎                 | 400/2500 [04:44<15:22,  2.28it/s, (Warmup)]






11:39:23 - cmdstanpy - INFO - CmdStan done processing.
11:39:23 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: lkj_corr_cholesky_lpdf: Random variable[3] is 0, but must be positive! (in 'Model_7.stan', line 62, column 2 to column 41)
	Exception: lkj_corr_cholesky_lpdf: Random variable[3] is 0, but must be positive! (in 'Model_7.stan', line 62, column 2 to column 41)
	Exception: lkj_corr_cholesky_lpdf: Random variable[3] is 0, but must be positive! (in 'Model_7.stan', line 62, column 2 to column 41)
	Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'Model_7.stan', line 75, column 2 to column 37)
	Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'Model_7.stan', line 75, column 2 to column 37)
	Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'Model_7.stan', line 75, column 2 to column 37)
	Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must

11:39:25 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 1 had 28 divergent transitions (2.8%)
	Chain 2 had 98 divergent transitions (9.8%)
	Chain 3 had 1 divergent transitions (0.1%)
	Chain 4 had 2 divergent transitions (0.2%)
	Use the "diagnose()" method on the CmdStanMCMC object to see further information.


Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
129 of 4000 (3.23%) transitions ended with a divergence.
These divergent transitions indicate that HMC is not fully able to explore the posterior distribution.
Try increasing adapt delta closer to 1.
If this doesn't remove all divergences, try to reparameterize the model.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory.

Rank-normalized split effective sample size satisfactory for all parameters.

The following parameters had rank-normalized split R-hat greater than 1.01:
  mu_cluster[1,1], mu_cluster[3,1], mu_cluster[1,2], mu_cluster[3,2], mu_cluster[1,3], mu_cluster[3,3], mu_cluster[1,4], mu_cluster[3,4], mu_cluster[1,5], mu_cluster[3,5], v[1], v[2], v[3], v[4], v[5], v[6], v[7], v[8], v[9], v[10], v[11], v[12], v[13], v[14], v[15], v[16], v[17], v[18], v[19], alpha, log_theta[1], log_theta[2], log_theta[3], log_theta[4], 

In [17]:
print(fit.diagnose())
summary_df = fit.summary()
summary_df.head(50)

Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
129 of 4000 (3.23%) transitions ended with a divergence.
These divergent transitions indicate that HMC is not fully able to explore the posterior distribution.
Try increasing adapt delta closer to 1.
If this doesn't remove all divergences, try to reparameterize the model.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory.

Rank-normalized split effective sample size satisfactory for all parameters.

The following parameters had rank-normalized split R-hat greater than 1.01:
  mu_cluster[1,1], mu_cluster[3,1], mu_cluster[1,2], mu_cluster[3,2], mu_cluster[1,3], mu_cluster[3,3], mu_cluster[1,4], mu_cluster[3,4], mu_cluster[1,5], mu_cluster[3,5], v[1], v[2], v[3], v[4], v[5], v[6], v[7], v[8], v[9], v[10], v[11], v[12], v[13], v[14], v[15], v[16], v[17], v[18], v[19], alpha, log_theta[1], log_theta[2], log_theta[3], log_theta[4], 

,Mean,MCSE,StdDev,MAD,5%,50%,95%,ESS_bulk,ESS_tail,ESS_bulk/s,R_hat
lp__,10345.400000,0.555844,15.789700,16.001700,10318.600000,10345.700000,10370.900000,808.812,1511.17,4.22849,1.003190
"beta[1,1]",-0.001399,0.000041,0.002746,0.002720,-0.005859,-0.001453,0.003112,4617.460,3264.47,24.14020,1.000210
"beta[1,2]",0.006918,0.000054,0.003621,0.003594,0.000989,0.006936,0.012828,4492.060,3141.95,23.48460,1.000010
"beta[1,3]",0.026294,0.000232,0.011807,0.011724,0.007034,0.026219,0.046035,2640.850,2791.26,13.80640,1.001070
"beta[1,4]",0.000654,0.000081,0.004262,0.004302,-0.006317,0.000572,0.007817,2838.420,3078.34,14.83930,1.000550
"beta[1,5]",0.003550,0.000026,0.001605,0.001622,0.000909,0.003579,0.006135,3781.940,3209.89,19.77210,1.000150
"beta[2,1]",-0.003570,0.000076,0.002952,0.002940,-0.008404,-0.003502,0.001356,1520.490,2310.44,7.94917,1.002770
"beta[2,2]",0.002748,0.000115,0.003957,0.003999,-0.003866,0.002846,0.009094,1183.360,1641.85,6.18660,1.002890
"beta[2,3]",0.098000,0.000358,0.012600,0.012443,0.077133,0.098069,0.118848,1259.580,1999.67,6.58509,1.000740
"beta[2,4]",0.018045,0.000138,0.004714,0.004607,0.010314,0.018003,0.026018,1189.900,1933.25,6.22083,1.001270


In [18]:
with open("Model_7.pkl", "wb") as f:
    pickle.dump(fit, f)